# Predicting House Sale Prices

In this project we focus on implementing a linear regression model in order to predict house prices on a data set from the city of Ames, Iowa. This data is from the years 2006-2010.

The first thing we want to do is lay out the functions we will be using.

In [126]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.model_selection import KFold

In [127]:
df = pd.read_csv("AmesHousing.tsv", delimiter= "\t")
pd.options.display.max_columns = 999

In [128]:
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numerical_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    
    features = numerical_train.columns.drop(['SalePrice'])
    
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    
    prediction = lr.predict(test[features])
    
    rmse = mean_squared_error(test['SalePrice'],prediction)**0.5
    return rmse


transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

These are the base functions of the model. We modify these to inlude selecting parts of the data set to use, cleaning the data set, and adding new features.

# Feature Engineering

In [129]:
df

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
5,6,527105030,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,195500
6,7,527127150,120,RL,41.0,4920,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,2001,2001,Gable,CompShg,CemntBd,CmentBd,None,0.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,616.0,Unf,0.0,722.0,1338.0,GasA,Ex,Y,SBrkr,1338,0,0,1338,1.0,0.0,2,0,2,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,582.0,TA,TA,Y,0,0,170,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,213500
7,8,527145080,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal,191500
8,9,527146030,120,RL,39.0,5389,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1995,1996,Gable,CompShg,CemntBd,CmentBd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,1180.0,Unf,0.0,415.0,1595.0,GasA,Ex,Y,SBrkr,1616,0,0,1

This is our dataset. We need to now do a couple of things. We need to remove features we don't want to use in the model. These would be columns with a lot of missing values or columns that leak information about the sale (such as the year that the sale happened).

In creating our function: we want to:
   - remove features that we don't want to use in the model, just based on the number of missing values or data leakage
   - transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc)
   - create new features by combining other features

In [130]:
#less than 5% NaN and over 5% NaN
# edit all changes then input into function. ex input transform to return cleaned df, etc

# Identify null values
null_series = df.isnull().sum()

# Isolate where columns have more than 5% null values
five_percent_nulls = null_series[null_series > (.05*len(df))]

#Drop these isolated columns from the dataframe
df = df.drop(five_percent_nulls.index, axis=1)

Now that the dataframe only contains columns with less than 5% null values, we can take another step in the cleaning process and:
    - Drop text columns with one or more missing values
    - Fill numerical column null values with the mode of the column.

In [131]:
# Isolate text datatype columns to see missing values
text_missing_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

# Isolate the columns that have missing text
drop_text_col = text_missing_counts[text_missing_counts > 0]

# Drop the isolated columns from the dataframe
df = df.drop(drop_text_col.index, axis=1)

In [132]:
# Next we isolate the numerical columns
numeric = df.select_dtypes(include=['int','float']).isnull().sum()

# We then fill these values in with the mode. Note we isolate the columns with less than 5% null
numeric = numeric[(numeric < .05*len(df)) & (numeric > 0)].sort_values()

df = df.fillna(stats.mode)

In [133]:
# We can verify that there are no null values remaining
df.isnull().sum().value_counts()

0    64
dtype: int64

We can also add features that can help us in our analysis

In [134]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [135]:
yrs_since_remod = df['Yr Sold'] - df['Year Remod/Add']
yrs_since_remod[yrs_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [136]:
# We can create new columns

df['Years_before_sale'] = df['Yr Sold'] - df['Year Built']
df['Years since remod'] = df['Yr Sold'] - df['Year Remod/Add']

In [137]:
# We can drop the rows with negative values for these columns
#df= df.drop([1702,2180,2181],axis=0)

# No longer need original year columns
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

In [138]:
# Lastly, lets drop columns that aren't useful for our ML model and ones that may leak data

## Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

We can now redo our transform_features function we created at the beginning to include all of these features we just put together.

In [139]:
def transform_features(df):
    null_series = df.isnull().sum()
    five_percent_nulls = null_series[null_series > (.05*len(df))]
    df = df.drop(five_percent_nulls.index, axis=1)
    
    text_missing_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_text_col = text_missing_counts[text_missing_counts > 0]
    df = df.drop(drop_text_col.index, axis=1)
    
    numeric = df.select_dtypes(include=['int','float']).isnull().sum()
    numeric = numeric[(numeric < .05*len(df)) & (numeric > 0)].sort_values()
    df = df.fillna(stats.mode)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    yrs_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years_before_sale'] = df['Yr Sold'] - df['Year Built']
    df['Years since remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df= df.drop([1702,2180,2181],axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    
    df = df.drop(["PID", "Order"], axis=1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numerical_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer','float'])
    
    
    features = numerical_train.columns.drop(['SalePrice'])
    
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    
    prediction = lr.predict(test[features])
    
    rmse = mean_squared_error(test['SalePrice'],prediction)**0.5
    return rmse

df= pd.read_csv("AmesHousing.tsv", delimiter= "\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.36731241307

Comparing the rmse from the initial set of functions we can see we have already improved a little bit. 

# Feature Selection

Now that we have cleaned and transformed a lot of the features in the data set, it's time to move on to feature selection for numerical features.

What we want to do is select the numerical features, these are ones that we can directly coorelate with our target column, SalePrice. We keep only strongly correlated columns. 
We also want to select the nominal columns, or columns where there is no given order, and see how many unique values are among these. We then drop ones with high uniqueness to reduce variability. 

Lastly, we isolate these columns, convert them to categorical data type, and add them back into the data frame using dummy variables.

In [140]:
numerical_df = transform_df.select_dtypes(include=['int', 'float'])

In [141]:
# We take the numerical columns and correlate them with Sale Price
corr_map = numerical_df.corr()['SalePrice'].abs().sort_values()
corr_map

Misc Val             0.019273
3Ssn Porch           0.032268
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Lot Area             0.267520
2nd Flr SF           0.269601
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Years since remod    0.534985
Full Bath            0.546118
Years_before_sale    0.558979
1st Flr SF           0.635185
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [142]:
# Lets only keep the features with a correlation values of 0.4 and higher

strong_corrs = corr_map[corr_map > 0.4]

transform_df = transform_df.drop(corr_map[corr_map < 0.4].index, axis=1)

In [143]:
strong_corrs

Fireplaces           0.474831
TotRms AbvGrd        0.498574
Years since remod    0.534985
Full Bath            0.546118
Years_before_sale    0.558979
1st Flr SF           0.635185
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

The next thing we want to do is identify the columns we want converted to the categorical data type.
All of the columns that can be categorized as nominal variables are candidates for being converted to categorical.

In [144]:
nominal_cols = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

# We identify the nominal columns that we had in the transform dataframe
transform_cat_col = []
for col in nominal_cols:
    if col in transform_df:
        transform_cat_col.append(col)

# We also want to see how many unique values these columns have
unique_count = transform_df[transform_cat_col].apply(lambda col: len(col.value_counts())).sort_values()

# We can also experiment with dropping columns with uniqueness counts over 10 (an arbitrary number)
non_unique = unique_count[unique_count > 10].index
transform_df = transform_df.drop(non_unique,axis=1)

In [145]:
# Next we can select the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')

# Finally we want to create dummy columns and then add these back into the dataframe

dummies = pd.get_dummies(transform_df.select_dtypes(include=['category']))

transform_df = pd.concat([transform_df,dummies],axis=1).drop(text_cols,axis=1)

Now we update the select_features function. 
Just like we did with the transform features function, we take the above code and use it to update the select_features function


In [164]:
def transform_features(df):
    null_series = df.isnull().sum()
    five_percent_nulls = null_series[null_series > (.05*len(df))]
    df = df.drop(five_percent_nulls.index, axis=1)
    
    text_missing_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_text_col = text_missing_counts[text_missing_counts > 0]
    df = df.drop(drop_text_col.index, axis=1)
    
    numeric = df.select_dtypes(include=['int','float']).isnull().sum()
    numeric = numeric[(numeric < .05*len(df)) & (numeric > 0)].sort_values()
    df = df.fillna(stats.mode)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    yrs_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years_before_sale'] = df['Yr Sold'] - df['Year Built']
    df['Years since remod'] = df['Yr Sold'] - df['Year Remod/Add']
    df= df.drop([1702,2180,2181],axis=0)
    df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)
    
    df = df.drop(["PID", "Order"], axis=1)
    df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    
    return df

def select_features(df,coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    corr_map = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(corr_map[corr_map < coeff_threshold].index,axis=1)
    
    nominal_cols = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

    transform_cat_col = []
    for col in nominal_cols:
        if col in df:
            transform_cat_col.append(col)

    unique_count = df[transform_cat_col].apply(lambda col: len(col.value_counts())).sort_values()

    non_unique = unique_count[unique_count > 10].index
    df = df.drop(non_unique,axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    
    dummies = pd.get_dummies(df.select_dtypes(include=['category']))

    transform_df = pd.concat([df,dummies],axis=1).drop(text_cols,axis=1)
    
    
    return df

def train_and_test(df, k=0):
    train = df[:1460]
    test = df[1460:]
    
    numerical_df = df.select_dtypes(include=['integer','float'])
    features = numerical_df.columns.drop(['SalePrice'])
    lr = LinearRegression()
    
    if k == 0:
        lr.fit(train[features], train['SalePrice'])
        prediction = lr.predict(test[features])
        rmse = mean_squared_error(test['SalePrice'],prediction)**0.5
        return rmse
    if k == 1:
        shuffled_df = df.sample(frac=1)
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train['SalePrice'])
        prediction_one = lr.predict(test[features])
        rmse_one = mean_squared_error(test['SalePrice'],prediction_one)**0.5
        
        lr.fit(test[features], test['SalePrice'])
        prediction_two = lr.predict(train[features])
        rmse_two = mean_squared_error(train['SalePrice'], prediction_two)**0.5
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    
    
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            
        lr.fit(train[features],train['SalePrice'])
        prediction = lr.predict(test[features])
        rmse = mean_squared_error(test['SalePrice'],prediction)**0.5
        rmse_values.append(rmse)
        
        avg_rmse = np.mean([rmse_values])
        
        return avg_rmse
        
        
        
df= pd.read_csv("AmesHousing.tsv", delimiter= "\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df,k=4)

rmse

32390.993333826256

We finally add in kFold cross validation in order as a resampling procedure for our model. This leaves us with a final rmse of about 32390.

There is a lot that can be added to improve this rmse. These topics will be revisited in the next project and this project will be redone to try and reduce this even further. 

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years_before_sale,Years since remod
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,Hip,CompShg,BrkFace,Plywood,112,TA,TA,CBlock,639,0,441,1080,GasA,Fa,Y,1656,0,0,1656,1,0,1,0,3,1,TA,7,Typ,2,2,528,P,210,62,0,0,0,0,0,215000,50,50
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,Gable,CompShg,VinylSd,VinylSd,0,TA,TA,CBlock,468,144,270,882,GasA,TA,Y,896,0,0,896,0,0,1,0,2,1,TA,5,Typ,0,1,730,Y,140,0,0,0,120,0,0,105000,49,49
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,Hip,CompShg,Wd Sdng,Wd Sdng,108,TA,TA,CBlock,923,0,406,1329,GasA,TA,Y,1329,0,0,1329,0,0,1,1,3,1,Gd,6,Typ,0,1,312,Y,393,36,0,0,0,0,12500,172000,52,52
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,Hip,CompShg,BrkFace,BrkFace,0,Gd,TA,CBlock,1065,0,1045,2110,GasA,Ex,Y,2110,0,0,2110,1,0,2,1,3,1,Ex,8,Typ,2,2,522,Y,0,0,0,0,0,0,0,244000,42,42
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,Gable,CompShg,VinylSd,VinylSd,0,TA,TA,PConc,791,0,137,928,GasA,Gd,Y,928,701,0,1629,0,0,2,1,3,1,TA,6,Typ,1,2,482,Y,212,34,0,0,0,0,0,189900,13,12
5,60,RL,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,Gable,CompShg,VinylSd,VinylSd,20,TA,TA,PConc,602,0,324,926,GasA,Ex,Y,926,678,0,1604,0,0,2,1,3,1,Gd,7,Typ,1,2,470,Y,360,36,0,0,0,0,0,195500,12,12
6,120,RL,4920,Pave,Reg,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,Gable,CompShg,CemntBd,CmentBd,0,Gd,TA,PConc,616,0,722,1338,GasA,Ex,Y,1338,0,0,1338,1,0,2,0,2,1,Gd,6,Typ,0,2,582,Y,0,0,170,0,0,0,0,213500,9,9
7,120,RL,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,Gable,CompShg,HdBoard,HdBoard,0,Gd,TA,PConc,263,0,1017,1280,GasA,Ex,Y,1280,0,0,1280,0,0,2,0,2,1,Gd,5,Typ,0,2,506,Y,0,82,0,0,144,0,0,191500,18,18
8,120,RL,5389,Pave,IR1,Lvl,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,Gable,CompShg,CemntBd,CmentBd,0,Gd,TA,PConc,1180,0,415,1595,GasA,Ex,Y,1616,0,0,1616,1,0,2,0,2,1,Gd,5,Typ,1,2,608,Y,237,152,0,0,0,0,0,236500,15,14
9,60,RL,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,VinylSd,VinylSd,0,TA,TA,PConc,0,0,994,994,GasA,Gd,Y,1028,776,0,1804,0,0,2,1,3,1,Gd,7,Typ,1,2,442,Y,140,60,0,0,0,0,0,189000,11,11
